In [1]:
%%time
import malaya

CPU times: user 11.4 s, sys: 1.54 s, total: 12.9 s
Wall time: 16.7 s


In [2]:
string = 'xjdi ke, y u xsuke makan HUSEIN kt situ tmpt, i hate it. pelikle'
another = 'i mmg xske mknn HUSEIN kampng tempt, i love them. pelikle'

## Load basic normalizer

In [3]:
malaya.normalize.basic(string)

'xjdi ke kenapa awak xsuke makan Husein kt situ tmpt i hate it pelikle'

## Load fuzzy normalizer

In [4]:
malays = malaya.load_malay_dictionary()
normalizer = malaya.normalize.fuzzy(malays)

In [5]:
normalizer.normalize(string)

'tak jadi ke kenapa awak tak suka makan Husein kat situ tempat saya hate it pelik lah'

In [6]:
normalizer.normalize(another)

'saya memang tak saka makanan Husein kampung tempt saya love them pelik lah'

## Load spell normalizer

In [7]:
normalizer = malaya.normalize.spell(malays)

#### To list all selected words during normalize

In [8]:
normalizer.normalize(string,debug=True)

[(('judi', False), 86), (('adi', False), 67), (('di', False), 80), (('jadi', False), 86)] 

[(('tepu', False), 50), (('amput', False), 67), (('tamat', False), 67), (('empat', True), 67), (('tumit', False), 67), (('ampe', False), 50), (('tipu', False), 50), (('tat', False), 57), (('top', False), 57), (('tampu', False), 67), (('topi', False), 50), (('tepi', False), 50), (('tempat', False), 80), (('umut', False), 50), (('ampo', False), 50), (('timpa', False), 67), (('impi', False), 50), (('tempe', False), 67), (('tapa', False), 50), (('taat', False), 50), (('tepet', False), 67), (('umat', False), 50), (('tepat', False), 67), (('tut', False), 57), (('tumpat', True), 80), (('tuat', False), 50), (('tampi', True), 67), (('umpat', True), 67), (('temut', False), 67), (('emat', False), 50), (('ampit', False), 67), (('amit', False), 50), (('tempo', False), 67), (('tumpu', False), 67), (('tempa', False), 67), (('empu', False), 50), (('amat', False), 50), (('taut', False), 50), (('mat', False), 57)

'tak jadi ke kenapa awak tak suka makan Husein kat situ tempat saya hate it pelik lah'

## List available deep learning stemming models

In [9]:
malaya.normalize.available_deep_model()

['lstm', 'bahdanau', 'luong']

## Load deep learning

We experimenting a lot for `seq2seq` models, we try to do the best normalizer deep learning models.

In [10]:
normalizer = malaya.normalize.deep_model(malays, 'bahdanau')
print(normalizer.normalize(string))
normalizer.normalize(another)

jidiomik ke kenapa awak sukeesi makan Husein kat situ tempatmo saya hate it pelik lah


'saya memang sikeuoi maknnkano Husein kampanga tempt saya love them pelik lah'

In [11]:
normalizer = malaya.normalize.deep_model(malays, 'luong')
print(normalizer.normalize(string))
normalizer.normalize(another)

jadidilox ke kenapa awak sokeled makan Husein kat situ tampatgllah saya hate it pelik lah


'saya memang skeflleh makafnnloh Husein kampangja tempt saya love them pelik lah'

In [12]:
normalizer = malaya.normalize.deep_model(malays, 'lstm')
print(normalizer.normalize(string))
normalizer.normalize(another)

jajiodi ke kenapa awak sukeeia makan Husein kat situ tempatwa saya hate it pelik lah


'saya memang sekeoia makankari Husein kampangi tempt saya love them pelik lah'